Problem Statement. <br/>

A group of friends went on holiday and sometimes lent each other money. For example, Alice paid for Bill's lunch for $10. Then later Chris gave Alice $5 for a taxi ride. We can model each transaction as a tuple (x, y, z) which means person x gave person y $z. Assuming Alice, Bill, and Chris are person 0, 1, and 2 respectively (0, 1, 2 are the person's ID), the transactions can be represented as [[0, 1, 10], [2, 0, 5]]. <br/>
Given a list of transactions between a group of people, return the minimum number of transactions required to settle the debt. <br/>

Note: <br/>
    A transaction will be given as a tuple (x, y, z). Note that x ≠ y and z > 0.
    Person's IDs may not be linear, e.g. we could have the persons 0, 1, 2 or we could also have the persons 0, 2, 6.

Example 1: <br/>
Input: [[0,1,10], [2,0,5]] <br/>
Output: 2 <br/>
Explanation: <br/>
Person 0 gave person 1 10. <br/>
Person 2 gave person 0 5. <br/>
Two transactions are needed. One way to settle the debt is person 1 pays person 0 and 2 5 each.

Example 2: <br/>
Input: [[0,1,10], [1,0,1], [1,2,5], [2,0,5]] <br/>
Output: 1 <br/>
Explanation: <br/>
Person 0 gave person 1 10. <br/>
Person 1 gave person 0 1. <br/>
Person 1 gave person 2 5. <br/>
Person 2 gave person 0 5. <br/>
Therefore, person 1 only need to give person 0 4, and all debt is settled.

# DFS and Bracktracking - O(N * 2^N) runtime, O(2^N) space

In [1]:
from typing import List
from collections import defaultdict

class Solution:
    def minTransfers(self, transactions: List[List[int]]) -> int:
        # Compute net profit for every person.
        bal = defaultdict(int)
        for t in transactions:
            bal[t[0]] += t[2]
            bal[t[1]] -= t[2]
            
        # Preserve unsettled people only.
        netProfit = [bal[k] for k in bal if bal[k] != 0]
        
        return self.traverse(netProfit, 0, 0)
    
    def traverse(self, netProfit, startIdx, numTrans):
        # Skip settled people.
        while startIdx < len(netProfit) and netProfit[startIdx] == 0:
            startIdx += 1
        if startIdx + 1 >= len(netProfit):
            return numTrans
        else:
            for i in range(startIdx + 1, len(netProfit)):
                # Greedy condition.
                if netProfit[startIdx] + netProfit[i] == 0:
                    netProfit[i] += netProfit[startIdx]
                    minNumTrans = self.traverse(netProfit, startIdx + 1, numTrans + 1)
                    netProfit[i] -= netProfit[startIdx]
                    return minNumTrans
            minNumTrans = float('inf')
            for i in range(startIdx + 1, len(netProfit)):
                # Non-greedy condition for possible closing out in the future.
                if netProfit[startIdx] * netProfit[i] < 0:
                    netProfit[i] += netProfit[startIdx]
                    minNumTrans = min(minNumTrans, self.traverse(netProfit, startIdx + 1, numTrans + 1))
                    netProfit[i] -= netProfit[startIdx]
            return minNumTrans

# Cleaner but slicing takes more time - O(N * 2^N) runtime, O(2^N) space

In [3]:
from typing import List
from collections import defaultdict

class Solution:
    def minTransfers(self, transactions: List[List[int]]) -> int:
        bal = defaultdict(int)
        for f, t, m in transactions:
            bal[f] -= m
            bal[t] += m
        balances = [bal[k] for k in bal if bal[k] != 0]
        return self.dfs(balances)
        
    def dfs(self, b):
        if not b:   # base case
            return 0

        if not b[0]:   #skip ppl whose balance==0 
            return self.dfs(b[1:])

        minTrans = float('inf')
        for i in range(1, len(b)):
            if b[i] == -b[0]:   # greedy shortcut 
                return 1 + self.dfs(b[1:i] + [0] + b[i+1:])

            elif b[i] * b[0] < 0: 
                minTrans = min(minTrans, self.dfs(b[1:i] + [b[i]+b[0]] + b[i+1:])) 

        return minTrans + 1  

# Faster with Bits - O(N * 2^N) runtime, O(2^N) space

In [8]:
from typing import List
from collections import defaultdict

class Solution:
    def minTransfers(self, transactions: List[List[int]]) -> int:
        persons = defaultdict(int)
        for sender, receiver, amount in transactions:
            persons[sender] -= amount
            persons[receiver] += amount
        
        amounts = [amount for amount in persons.values() if amount != 0]
        
        N = len(amounts)
        dp = [0] * (2**N) # dp[mask] = number of sets whose sum = 0
        sums = [0] * (2**N) # sums[mask] = sum of numbers in mask
        
        for mask in range(2**N):
            set_bit = 1
            for b in range(N):
                if mask & set_bit == 0:
                    nxt = mask | set_bit
                    sums[nxt] = sums[mask] + amounts[b]
                    if sums[nxt] == 0: dp[nxt] = max(dp[nxt], dp[mask] + 1)
                    else: dp[nxt] = max(dp[nxt], dp[mask])
                set_bit <<= 1
        
        return N - dp[-1]   

In [9]:
instance = Solution()
instance.minTransfers([[0,1,10], [1,0,1], [1,2,5], [2,0,5]] )

1